# Soil Analysis Using CNN

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#path to the dataset
dir_black = os.path.join('Soil types/Black Soil')
dir_Cinder = os.path.join('Soil types/Cinder Soil')
dir_Laterite = os.path.join('Soil types/Laterite Soil')
dir_peat = os.path.join('Soil types/Peat Soil')
dir_yellow = os.path.join('Soil types/Yellow Soil')

In [7]:
image_size = 220
batch_size = 10
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

In [8]:
# Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        'Soil types/', 
        target_size=(200, 200),
        batch_size = batch_size,
        classes = [ 'Black Soil','Cinder Soil', 'Laterite Soil','Peat Soil','Yellow Soil'],
       class_mode='categorical')

Found 29 images belonging to 5 classes.


In [9]:
for image_batch, label_batch in train_generator:
  break
  image_batch.shape, label_batch.shape

In [10]:
print (train_generator.class_indices)

{'Black Soil': 0, 'Cinder Soil': 1, 'Laterite Soil': 2, 'Peat Soil': 3, 'Yellow Soil': 4}


In [21]:
# Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(220, 220, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 218, 218, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 109, 109, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 107, 107, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 51, 51, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 23, 23, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 229,285 (895.64 KB)

 Trainable params: 229,285 (895.64 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [14]:
total_sample = train_generator.n
n_epochs = 50

In [15]:
history = model.fit(
        train_generator, 
        steps_per_epoch = int(total_sample/batch_size),  
        epochs = n_epochs,
        verbose = 1)

Epoch 1/50


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - acc: 0.7333 - loss: 1.3514
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - acc: 1.0000 - loss: 3.9351e-04 
Epoch 3/50


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - acc: 1.0000 - loss: 4.0207e-04
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 1.0000 - loss: 7.4469e-04 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - acc: 1.0000 - loss: 4.1650e-04
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 1.0000 - loss: 1.1055e-04  
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - acc: 1.0000 - loss: 1.9048e-04
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 1.0000 - loss: 4.1817e-04  
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - acc: 1.0000 - loss: 1.0461e-04
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 1.0000 - loss: 3.5087e-04 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - acc: 1.0000 - loss: 1.1490e-04
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 1.0000 - loss: 2.3471e-04  
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - acc: 1.0000 - loss: 7.8435e-05
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 1.0000 - loss: 1.8664e-04  
Epoch 15/5

In [19]:
plt.figure(figsize=(8, 4))  # Adjust the figure size to your preference

# Setting a Seaborn style
sns.set_style("whitegrid")
# Plotting the data with improved aesthetics
plt.plot(range(1, n_epochs + 1), history['acc'], marker='o', color='black', linewidth=1, markersize=9)

# Title and labels with increased font sizes
plt.title("Training Accuracy with Epochs", fontsize=20)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Training Accuracy", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Customizing background color and tick colors
plt.gca().set_facecolor('#f5f5f5')  # Set background color
plt.tick_params(axis='x', colors='gray')  # Set x-axis tick color
plt.tick_params(axis='y', colors='gray')  # Set y-axis tick color

# Showing the plot
plt.show()


TypeError: 'History' object is not subscriptable

<Figure size 800x400 with 0 Axes>

In [23]:
model.save('soil_analysis_V1.h5')

# End